# Prepare Proteomic Data - Intensities, REDS RBC Omics

1. Nemkov T, Stephenson D, Earley EJ, Keele GR, Hay A, Key A, Haiman ZB, Erickson C, Dzieciatkowska M, Reisz JA, Moore A, Stone M, Deng X, Kleinman S, Spitalnik SL, Hod EA, Hudson KE, Hansen KC, Palsson BO, Churchill GA, Roubinian N, Norris PJ, Busch MP, Zimring JC, Page GP, D'Alessandro A. Biological and genetic determinants of glycolysis: Phosphofructokinase isoforms boost energy status of stored red blood cells and transfusion outcomes. Cell Metab. 2024 Sep 3;36(9):1979-1997.e13. doi: 10.1016/j.cmet.2024.06.007. Epub 2024 Jul 3. PMID: 38964323; PMCID: PMC11374506.

2. D'Alessandro A, Culp-Hill R, Reisz JA, Anderson M, Fu X, Nemkov T, Gehrke S, Zheng C, Kanias T, Guo Y, Page G, Gladwin MT, Kleinman S, Lanteri M, Stone M, Busch M, Zimring JC; Recipient Epidemiology and Donor Evaluation Study-III (REDS-III). Heterogeneity of blood processing and storage additives in different centers impacts stored red blood cell metabolism as much as storage time: lessons from REDS-III-Omics. Transfusion. 2019 Jan;59(1):89-100. doi: 10.1111/trf.14979. Epub 2018 Oct 24. PMID: 30353560; PMCID: PMC6322946.

3. Josephson CD, Glynn S, Mathew S, Birch R, Bakkour S, Baumann Kreuziger L, Busch MP, Chapman K, Dinardo C, Hendrickson J, Hod EA, Kelly S, Luban N, Mast A, Norris P, Custer B, Sabino E, Sachais B, Spencer BR, Stone M, Kleinman S; National Heart, Lung, and Blood Institute (NHLBI) Recipient Epidemiology and Donor Evaluation Study-IV-Pediatric (REDS-IV-P). The Recipient Epidemiology and Donor Evaluation Study-IV-Pediatric (REDS-IV-P): A research program striving to improve blood donor safety and optimize transfusion outcomes across the lifespan. Transfusion. 2022 May;62(5):982-999. doi: 10.1111/trf.16869. Epub 2022 Apr 19. PMID: 35441384; PMCID: PMC9353062.

## Setup
### Import packages

In [1]:
import pandas as pd
from rbc_gem_utils import get_dirpath, show_versions
from rbc_gem_utils.util import AVOGADRO_NUMBER, ensure_iterable

# Show versions of notebook
show_versions()


Package Information
-------------------
rbc-gem-utils 0.0.2

Dependency Information
----------------------
beautifulsoup4                       4.13.4
bio                                   1.8.0
cobra                                0.29.1
depinfo                               2.2.0
gurobipy                             12.0.2
matplotlib                           3.10.3
matplotlib-venn                       1.1.2
memote                               0.17.0
networkx                              3.4.2
notebook                              7.4.2
openpyxl                              3.1.5
pandas                                2.2.3
pre-commit                            4.2.0
rbc-gem-utils[database,network,vis] missing
requests                             2.32.3
scikit-learn                          1.6.1
scipy                                1.15.3
seaborn                              0.13.2

Build Tools Information
-----------------------
pip          25.1
setuptools 78.1.1
wheel      0.45

## Set organism, dataset, and paths

In [2]:
organism = "Human"
dataset_name = "REDS_RBCOmics"
raw_data_dirpath = get_dirpath(use_temp="raw") / organism / dataset_name

# Ensure directory exists
processed_data_dirpath = get_dirpath(use_temp="processed") / organism / dataset_name
processed_data_dirpath.mkdir(exist_ok=True, parents=True)

## Set data value type and variables for columns keys 

In [3]:
protein_values_dtype = "Intensities"
sample_key = "SAMPLE ID"
donor_key = "PUBLIC DONOR ID"
time_key = "DAY"

time_abbrev = "D"

## Load RBC Proteomics
### Load protein data

In [4]:
df_protein_data = pd.read_csv(
    raw_data_dirpath / f"{dataset_name}_ProteinData.tsv",
    sep="\t",
    index_col=None,
)
# Check to see if expected columns are included. If so, then order columns as listed.
# Comes directly from UniProt if possible
df_protein_data = df_protein_data.loc[
    :,
    [
        "Entry",
        "Entry Name",
        "Protein",
        "Protein names",
        "Gene Names (primary)",
        "Length",
        "Mass",  # Should be in DA
    ],
]
# Sort the data via alphabetical order of protein IDs for consistency
df_protein_data = df_protein_data.set_index("Entry").sort_index()
df_protein_data.head()

,Entry Name,Protein,Protein names,Gene Names (primary),Length,Mass
Entry,,,,,,
A0A075B6I0,LV861_HUMAN,LV861,Immunoglobulin lambda variable 8-61,IGLV8-61,122,12814
A0A075B6I9,LV746_HUMAN,LV746,Immunoglobulin lambda variable 7-46,IGLV7-46,117,12468
A0A075B6J9,LV218_HUMAN,LV218,Immunoglobulin lambda variable 2-18,IGLV2-18,118,12412
A0A075B6K4,LV310_HUMAN,LV310,Immunoglobulin lambda variable 3-10,IGLV3-10,115,12441
A0A075B6K5,LV39_HUMAN,LV39,Immunoglobulin lambda variable 3-9,IGLV3-9,115,12332


#### Load proteomics and map to UniProt if necessary

In [5]:
df_proteomics = pd.read_csv(
    raw_data_dirpath / f"{dataset_name}_Protein{protein_values_dtype}.tsv",
    sep="\t",
    index_col=None,
)
original_ids_type = "Protein"

# Create sample IDs from donor and time points, then set as index
df_proteomics.index = pd.Index(
    df_proteomics[[donor_key, time_key]]
    .apply(lambda x: f"{x[donor_key]}_{time_abbrev}{x[time_key]}", axis=1)
    .values,
    name=sample_key,
)

# Transform Protein IDs to UniProt IDs
if original_ids_type != "uniprot" and any(
    df_proteomics.columns.isin(df_protein_data[original_ids_type])
):
    mapping_dict = df_protein_data.reset_index(drop=False)
    mapping_dict = mapping_dict.set_index(original_ids_type)[df_protein_data.index.name]
    mapping_dict = mapping_dict.to_dict()
    df_proteomics = df_proteomics.rename(mapping_dict, axis=1)

# Sort for consistency
df_proteomics = df_proteomics.sort_index(axis=0)[
    [donor_key, time_key] + list(df_protein_data.index)
]
donor_ids = df_proteomics[donor_key].unique()
timepoints = df_proteomics[time_key].unique()
print(f"Number of donors: {len(donor_ids)}")
print(f"Number of timepoints: {len(timepoints)}")
print(f"Number of expected samples: {len(donor_ids) * len(timepoints)}")
print(f"Number of actual samples: {len(df_proteomics)}")
print(
    f"Number of duplicated IDs: {len(df_proteomics[df_proteomics.index.duplicated()])}"
)
df_proteomics

Number of donors: 651
Number of timepoints: 3
Number of expected samples: 1953
Number of actual samples: 1920
Number of duplicated IDs: 5


,PUBLIC DONOR ID,DAY,A0A075B6I0,A0A075B6I9,A0A075B6J9,A0A075B6K4,A0A075B6K5,A0A075B6R2,A0A075B6S5,A0A075B6S9,...,Q9Y639,Q9Y666,Q9Y696,Q9Y6B6,Q9Y6B7,Q9Y6E0,Q9Y6I3,Q9Y6M4,Q9Y6M5,Q9Y6R7
SAMPLE ID,,,,,,,,,,,,,,,,,,,,,
S001_D10,S001,10,114.106690,61.188179,16.998806,1.0,240.439072,0.000000,8.726534,0.000000,...,4.806035,0.000000,44.643505,130.273163,0.000000,30.318407,22.992273,0.000000,38.119408,202.467422
S001_D23,S001,23,90.694664,18.855646,0.000000,0.0,234.999283,0.000000,30.381741,148.992706,...,3.503707,0.000000,28.674469,0.000000,0.000000,33.782383,159.744751,21.729137,38.118774,210.148560
S001_D42,S001,42,123.064743,49.431454,61.730526,0.0,221.155167,0.000000,25.704966,0.000000,...,7.449622,1.478644,26.478333,139.307129,7.024934,45.400738,19.826700,22.173162,28.154861,191.642059
S002_D10,S002,10,0.000000,0.000000,0.000000,0.0,267.438843,12.034687,24.897249,0.000000,...,12.975469,0.000000,53.475803,91.111374,0.000000,50.815300,0.000000,0.000000,113.735794,264.970459
S002_D23,S002,23,64.492546,0.000000,490.452606,0.0,518.987244,0.000000,0.000000,0.000000,...,10.578275,48.285789,48.756619,13.925104,0.000000,77.841980,0.000000,0.000000,129.577789,93.992775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S651_D10,S651,10,324.720825,0.000000,104.024376,0.0,324.189484,9.646808,6.394954,75.480797,...,3.162988,8.217668,32.108208,194.344620,0.000000,24.224831,26.392359,0.000000,30.559240,869.670654
S651_D23,S651,23,269.245667,0.000000,0.000000,0.0,288.051331,9.713434,25.288080,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,38.350945,7.902850,0.000000,41.832993,105.314491
S651_D23,S651,23,382.593780,13.661825,27.932926,0.0,229.929688,0.000000,24.017260,62.770485,...,1.055674,0.000000,6.279982,27.841568,0.000000,30.486551,32.314617,44.323257,29.898968,104.690201


### Load metadata corresponding to samples (optional)
#### Genotype data

In [6]:
try:
    df_genotypes = pd.read_csv(
        raw_data_dirpath / f"{dataset_name}_Genotypes.tsv",
        sep="\t",
        index_col=[donor_key],
    )
except FileNotFoundError:
    df_genotypes = pd.DataFrame([])
df_genotypes

,G6PD_V68M
PUBLIC DONOR ID,
S001,0.0
S002,0.0
S003,0.0
S004,0.0
S005,0.0
...,...
S647,0.0
S648,0.0
S649,0.0


#### Phenotype data

In [7]:
try:
    df_phenotypes = pd.read_csv(
        raw_data_dirpath / f"{dataset_name}_Phenotypes.tsv",
        sep="\t",
        index_col=[donor_key],
    )
except FileNotFoundError:
    df_phenotypes = pd.DataFrame([])
df_phenotypes

,AS,Gender,DONDB.ABO_RH,RBCOmics.Race.Ethnicity.Group,Age,BMI,Weight,Height,Hemolysis.volume,Hemolysis.hct,...,Adjusted.Osmotic.Hemolysis,Adjusted.Oxidative.Hemolysis,CBC.WBC,CBC.RBC,CBC.HGB,CBC.HCT,CBC.MCV,CBC.RDW,CBC.PLT,Ferritin
PUBLIC DONOR ID,,,,,,,,,,,,,,,,,,,,,
S001,AS1,M,O+,CAUCASIAN,33,25.724339,195,73,14.00,65.00,...,51.412936,NaN,5.56,5.33,15.5,46.60,87.40,12.7,274.0,19.0
S002,AS3,M,A+,HIGH,71,27.891291,200,71,14.00,67.00,...,72.000942,NaN,7.32,5.38,16.5,47.90,89.00,12.7,222.0,23.0
S003,AS1,F,O+,HIGH,51,23.128284,139,65,11.50,64.00,...,25.647109,NaN,5.67,4.38,14.2,41.70,95.20,12.1,293.0,17.0
S004,AS1,F,A+,CAUCASIAN,48,23.294675,140,65,12.50,62.00,...,12.702554,NaN,6.10,4.32,13.1,40.90,94.70,12.3,376.0,21.0
S005,AS1,M,A+,OTHER,52,36.042480,210,64,11.00,63.50,...,55.111380,NaN,7.52,5.81,15.9,47.80,82.30,14.4,414.0,93.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S647,AS1,M,O+,HISPANIC,31,30.128571,210,70,10.00,55.00,...,17.757095,57.301236,4.93,4.92,14.3,42.60,86.60,13.2,269.0,118.0
S648,AS1,M,B+,AFRAMRCN,36,47.988868,325,69,14.00,60.00,...,31.811182,52.646317,6.38,5.54,15.6,47.20,85.20,12.7,234.0,120.0
S649,AS3,F,A+,CAUCASIAN,61,23.170166,135,64,13.25,61.29,...,42.473012,NaN,4.30,4.24,13.5,41.00,96.70,13.1,223.0,17.0


##### Cut phenotype data into ranges if desired

In [8]:
cuts = {
    "BMI": [0, 25, 30, 40, 60],
    "Age": [0, 20, 40, 60, 80, 100],
}
if not df_phenotypes.empty:
    for col, bins in cuts.items():
        labels = []
        for idx in range(1, len(bins)):
            if idx == 1 or idx == len(bins) - 1:
                labels += [f"lt{bins[idx]:d}"]
            elif idx:
                labels += [f"gt{bins[idx - 1]:d}"]
            else:
                labels += [f"{bins[idx - 1]:d}to{bins[idx]:d}"]
        df_phenotypes[f"{col}_Range"] = pd.cut(
            df_phenotypes[col], bins=bins, labels=labels, right=False
        )
df_phenotypes

,AS,Gender,DONDB.ABO_RH,RBCOmics.Race.Ethnicity.Group,Age,BMI,Weight,Height,Hemolysis.volume,Hemolysis.hct,...,CBC.WBC,CBC.RBC,CBC.HGB,CBC.HCT,CBC.MCV,CBC.RDW,CBC.PLT,Ferritin,BMI_Range,Age_Range
PUBLIC DONOR ID,,,,,,,,,,,,,,,,,,,,,
S001,AS1,M,O+,CAUCASIAN,33,25.724339,195,73,14.00,65.00,...,5.56,5.33,15.5,46.60,87.40,12.7,274.0,19.0,gt25,gt20
S002,AS3,M,A+,HIGH,71,27.891291,200,71,14.00,67.00,...,7.32,5.38,16.5,47.90,89.00,12.7,222.0,23.0,gt25,gt60
S003,AS1,F,O+,HIGH,51,23.128284,139,65,11.50,64.00,...,5.67,4.38,14.2,41.70,95.20,12.1,293.0,17.0,lt25,gt40
S004,AS1,F,A+,CAUCASIAN,48,23.294675,140,65,12.50,62.00,...,6.10,4.32,13.1,40.90,94.70,12.3,376.0,21.0,lt25,gt40
S005,AS1,M,A+,OTHER,52,36.042480,210,64,11.00,63.50,...,7.52,5.81,15.9,47.80,82.30,14.4,414.0,93.0,gt30,gt40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S647,AS1,M,O+,HISPANIC,31,30.128571,210,70,10.00,55.00,...,4.93,4.92,14.3,42.60,86.60,13.2,269.0,118.0,gt30,gt20
S648,AS1,M,B+,AFRAMRCN,36,47.988868,325,69,14.00,60.00,...,6.38,5.54,15.6,47.20,85.20,12.7,234.0,120.0,lt60,gt20
S649,AS3,F,A+,CAUCASIAN,61,23.170166,135,64,13.25,61.29,...,4.30,4.24,13.5,41.00,96.70,13.1,223.0,17.0,lt25,gt60


#### Combine into one DataFrame for MetaData

In [9]:
print(f"Proteomics: {df_proteomics[donor_key].nunique()} donors")
print(
    f"  Genomics: {df_genotypes.dropna(how='all', axis=0).index.nunique() if not df_genotypes.empty else 0} donors"
)
print(
    f"Phenotypes: {df_phenotypes.dropna(how='all', axis=0).index.nunique() if not df_phenotypes.empty else 0} donors"
)


df_metadata = pd.concat((df_genotypes, df_phenotypes), axis=1).convert_dtypes()

if not df_metadata.empty:
    df_metadata = df_metadata.reset_index(drop=False)
    # Ensure only metadata corresponds to the available omics data
    if not df_metadata[donor_key].isin(df_proteomics[donor_key]).all():
        df_metadata = df_metadata[df_metadata[donor_key].isin(df_proteomics[donor_key])]

    # If time was not included in metadata, add as a part of index to ensure index matches samples
    if time_key and time_key not in df_metadata.index:
        df_metadata = (
            pd.concat(
                (
                    df_metadata,
                    pd.Series(
                        [list(df_proteomics[time_key].unique())]
                        * len(df_metadata.index),
                        index=df_metadata.index,
                        name=time_key,
                    ),
                ),
                axis=1,
            )
            .explode(time_key)
            .reset_index(drop=True)
        )
    # Create sample IDs from donor and time points, then set as index
    df_metadata.index = pd.Index(
        df_metadata[[donor_key, time_key]]
        .apply(lambda x: f"{x[donor_key]}_{time_abbrev}{x[time_key]}", axis=1)
        .values,
        name=sample_key,
    )
    print(f"\nFinal data: {df_metadata[donor_key].nunique()} donors")
    df_metadata = df_metadata.drop([donor_key, time_key], axis=1)
else:
    print(f"\nFinal Meta: 0 donors")

df_metadata.head()

Proteomics: 651 donors
  Genomics: 639 donors
Phenotypes: 651 donors

Final data: 651 donors


,G6PD_V68M,AS,Gender,DONDB.ABO_RH,RBCOmics.Race.Ethnicity.Group,Age,BMI,Weight,Height,Hemolysis.volume,...,CBC.WBC,CBC.RBC,CBC.HGB,CBC.HCT,CBC.MCV,CBC.RDW,CBC.PLT,Ferritin,BMI_Range,Age_Range
SAMPLE ID,,,,,,,,,,,,,,,,,,,,,
S001_D10,0,AS1,M,O+,CAUCASIAN,33,25.724339,195,73,14.0,...,5.56,5.33,15.5,46.6,87.4,12.7,274.0,19.0,gt25,gt20
S001_D23,0,AS1,M,O+,CAUCASIAN,33,25.724339,195,73,14.0,...,5.56,5.33,15.5,46.6,87.4,12.7,274.0,19.0,gt25,gt20
S001_D42,0,AS1,M,O+,CAUCASIAN,33,25.724339,195,73,14.0,...,5.56,5.33,15.5,46.6,87.4,12.7,274.0,19.0,gt25,gt20
S002_D10,0,AS3,M,A+,HIGH,71,27.891291,200,71,14.0,...,7.32,5.38,16.5,47.9,89.0,12.7,222.0,23.0,gt25,gt60
S002_D23,0,AS3,M,A+,HIGH,71,27.891291,200,71,14.0,...,7.32,5.38,16.5,47.9,89.0,12.7,222.0,23.0,gt25,gt60


### Get MCH per sample

In [10]:
# Provide in picograms. Set as None to use metadata if provided
mch_sample_value = None
if mch_sample_value is None:
    try:
        df_MCH_per_sample = pd.read_csv(
            raw_data_dirpath / f"{dataset_name}_Phenotypes.tsv",
            sep="\t",
            index_col=None,
        )

    except FileNotFoundError:
        raise ValueError(
            "Cannot determine MCH. No phenotype data provided and a default value is not provided"
        )

    # Ensure only metadata corresponds to the available omics data
    if not df_MCH_per_sample[donor_key].isin(df_proteomics[donor_key]).all():
        df_MCH_per_sample = df_MCH_per_sample[
            df_MCH_per_sample[donor_key].isin(df_proteomics[donor_key])
        ]

    if "CBC.MCH" not in df_MCH_per_sample.columns:
        if all([x in df_MCH_per_sample.columns for x in ["CBC.HGB", "CBC.RBC"]]):
            df_MCH_per_sample["CBC.MCH"] = (
                df_MCH_per_sample["CBC.HGB"] / df_MCH_per_sample["CBC.RBC"]
            ) * 10
        elif all([x in df_MCH_per_sample.columns for x in ["CBC.MCHC", "CBC.MCV"]]):
            df_MCH_per_sample["CBC.MCH"] = (
                df_MCH_per_sample["CBC.MCHC"] * df_MCH_per_sample["CBC.MCV"]
            ) / 100
        else:
            raise ValueError(
                "Cannot determine MCH, one of the following combinations is needed: (CBC.HGB and CBC.RBC) or (CBC.MCHC and CBC.MCV)"
            )
    df_MCH_per_sample = df_MCH_per_sample.set_index(donor_key)["CBC.MCH"]
    n_missing = len(df_MCH_per_sample[df_MCH_per_sample.isna()])
    print(f"Missing values for {n_missing} samples.")
    print(f"Mean MCH in pg: {df_MCH_per_sample.mean():.2f}")
    df_MCH_per_sample = df_MCH_per_sample.fillna(df_MCH_per_sample.mean())
else:
    print("Using default MCH value provided for all samples")
    df_MCH_per_sample = pd.Series(
        [mch_sample_value] * df_proteomics[donor_key].nunique(),
        index=pd.Index(df_proteomics[donor_key].unique(), name=donor_key),
        name="CBC.MCH",
    )
    print(f"Mean MCH in pg: {mch_sample_value:.2f}")

df_MCH_per_sample = df_MCH_per_sample.reset_index(drop=False)
# If time was not included in metadata, add as a part of index to ensure index matches samples
if time_key and time_key not in df_MCH_per_sample.index:
    df_MCH_per_sample = (
        pd.concat(
            (
                df_MCH_per_sample,
                pd.Series(
                    [list(df_proteomics[time_key].unique())]
                    * len(df_MCH_per_sample.index),
                    index=df_MCH_per_sample.index,
                    name=time_key,
                ),
            ),
            axis=1,
        )
        .explode(time_key)
        .reset_index(drop=True)
    )
# Create sample IDs from donor and time points, then set as index
df_MCH_per_sample.index = pd.Index(
    df_MCH_per_sample[[donor_key, time_key]]
    .apply(lambda x: f"{x[donor_key]}_{time_abbrev}{x[time_key]}", axis=1)
    .values,
    name=sample_key,
)
df_MCH_per_sample.head()

Missing values for 12 samples.
Mean MCH in pg: 29.51


,PUBLIC DONOR ID,CBC.MCH,DAY
SAMPLE ID,,,
S001_D10,S001,29.080675,10
S001_D23,S001,29.080675,23
S001_D42,S001,29.080675,42
S002_D10,S002,30.669145,10
S002_D23,S002,30.669145,23


### Remove duplicated IDs in samples

In [11]:
df_proteomics = df_proteomics[~df_proteomics.index.duplicated(keep=False)]
df_MCH_per_sample = df_MCH_per_sample.loc[df_proteomics.index]
df_proteomics

,PUBLIC DONOR ID,DAY,A0A075B6I0,A0A075B6I9,A0A075B6J9,A0A075B6K4,A0A075B6K5,A0A075B6R2,A0A075B6S5,A0A075B6S9,...,Q9Y639,Q9Y666,Q9Y696,Q9Y6B6,Q9Y6B7,Q9Y6E0,Q9Y6I3,Q9Y6M4,Q9Y6M5,Q9Y6R7
SAMPLE ID,,,,,,,,,,,,,,,,,,,,,
S001_D10,S001,10,114.106690,61.188179,16.998806,1.0,240.439072,0.000000,8.726534,0.000000,...,4.806035,0.000000,44.643505,130.273163,0.000000,30.318407,22.992273,0.000000,38.119408,202.467422
S001_D23,S001,23,90.694664,18.855646,0.000000,0.0,234.999283,0.000000,30.381741,148.992706,...,3.503707,0.000000,28.674469,0.000000,0.000000,33.782383,159.744751,21.729137,38.118774,210.148560
S001_D42,S001,42,123.064743,49.431454,61.730526,0.0,221.155167,0.000000,25.704966,0.000000,...,7.449622,1.478644,26.478333,139.307129,7.024934,45.400738,19.826700,22.173162,28.154861,191.642059
S002_D10,S002,10,0.000000,0.000000,0.000000,0.0,267.438843,12.034687,24.897249,0.000000,...,12.975469,0.000000,53.475803,91.111374,0.000000,50.815300,0.000000,0.000000,113.735794,264.970459
S002_D23,S002,23,64.492546,0.000000,490.452606,0.0,518.987244,0.000000,0.000000,0.000000,...,10.578275,48.285789,48.756619,13.925104,0.000000,77.841980,0.000000,0.000000,129.577789,93.992775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S648_D42,S648,42,77.322556,0.000000,0.000000,0.0,519.996765,0.000000,22.033518,0.000000,...,9.065331,0.000000,68.654137,0.000000,0.000000,43.890713,0.000000,0.000000,62.910065,98.207466
S649_D10,S649,10,90.417908,0.000000,64.904564,0.0,121.021606,0.000000,8.485724,0.000000,...,4.922802,0.000000,18.535042,155.032333,0.000000,29.087191,19.722677,0.000000,35.296345,138.553513
S649_D42,S649,42,95.081497,0.000000,0.000000,0.0,86.734108,0.000000,4.888623,32.385452,...,5.490839,13.410266,23.362236,132.913025,0.000000,34.493198,15.165656,72.291283,39.360321,99.929260


### Get data subsets using operations

In [12]:
operations = [
    "mean",
    "median",
]
operation_dfs_proteomics = []
operation_dfs_MCH = []
fill_keys = set()


def group_data(df, operation, keys, columns, prefix_values=None, name_col=None):
    keys = ensure_iterable(keys)
    if not prefix_values:
        prefix_values = [""] * len(keys)
    if isinstance(prefix_values, dict):
        prefix_values = {k: prefix_values.get(k, "") for k in keys}
    else:
        prefix_values = dict(zip(keys, prefix_values))

    df = df.groupby(keys, as_index=False, observed=False)[columns]
    df = getattr(df, operation.lower())()
    labels = df[keys].apply(
        lambda x: "_".join([f"{prefix_values[key]}{x[key]}" for key in keys]),
        axis=1,
    )
    df[name_col] = [f"{operation.capitalize()}_{value}" for value in labels]
    return df

#### Group by donor only

In [13]:
# keys = [donor_key]
# prefix_values = {}

# operation_dfs_proteomics += [
#     group_data(
#         df_proteomics.reset_index(drop=False),
#         operation,
#         keys=keys,
#         columns=list(df_protein_data.index),
#         prefix_values=prefix_values,
#         name_col=sample_key,
#     )
#     for operation in operations
# ]

# operation_dfs_MCH += [
#     group_data(
#         df_MCH_per_sample.reset_index(drop=False),
#         operation,
#         keys=keys,
#         columns=["CBC.MCH"],
#         prefix_values=prefix_values,
#         name_col=sample_key,
#     )
#     for operation in operations
# ]

#### Group by time only

In [14]:
keys = [time_key]
prefix_values = {time_key: time_abbrev}

operation_dfs_proteomics += [
    group_data(
        df_proteomics.reset_index(drop=False),
        operation,
        keys=keys,
        columns=list(df_protein_data.index),
        prefix_values=prefix_values,
        name_col=sample_key,
    )
    for operation in operations
]

operation_dfs_MCH += [
    group_data(
        df_MCH_per_sample.reset_index(drop=False),
        operation,
        keys=keys,
        columns=["CBC.MCH"],
        prefix_values=prefix_values,
        name_col=sample_key,
    )
    for operation in operations
]

#### Group by metadata only

In [15]:
keys_prefixes = {
    "G6PD_V68M": "G6PD_V68M",
    # "Gender": "Sex",
    # "BMI_Range": "BMI",
    # "Age_Range": "Age",
}
keys_prefixes = {k: f"{v}_" for k, v in keys_prefixes.items()}
for key, prefix in keys_prefixes.items():

    operation_dfs_proteomics += [
        group_data(
            pd.merge(
                df_proteomics,
                df_metadata,
                left_index=True,
                right_index=True,
                how="left",
            ).reset_index(drop=False),
            operation,
            keys=[key],
            columns=list(df_protein_data.index),
            prefix_values={key: prefix},
            name_col=sample_key,
        )
        for operation in operations
    ]

    operation_dfs_MCH += [
        group_data(
            pd.merge(
                df_MCH_per_sample,
                df_metadata,
                left_index=True,
                right_index=True,
                how="left",
            ).reset_index(drop=False),
            operation,
            keys=[key],
            columns=["CBC.MCH"],
            prefix_values={key: prefix},
            name_col=sample_key,
        )
        for operation in operations
    ]

### Add to DataFrames

In [16]:
try:
    df_proteomics_op = pd.concat(operation_dfs_proteomics, axis=0).drop_duplicates()
except (KeyError, ValueError):
    df_proteomics_final = df_proteomics[df_protein_data.index].copy()
else:
    df_proteomics_final = pd.concat(
        (df_proteomics.reset_index(drop=False), df_proteomics_op), axis=0
    )
    df_proteomics_final = df_proteomics_final.set_index(sample_key)[
        df_protein_data.index
    ]

try:
    df_MCH_op = pd.concat(operation_dfs_MCH, axis=0).drop_duplicates()
except (KeyError, ValueError):
    df_MCH_final = df_MCH_per_sample["CBC.MCH"].copy()
else:
    df_MCH_final = pd.concat(
        (df_MCH_per_sample.reset_index(drop=False), df_MCH_op), axis=0
    )
    df_MCH_final = df_MCH_final.set_index(sample_key)["CBC.MCH"]

df_MCH_final.name = "MCH"
df_MCH_final
df_MCH_per_sample

,PUBLIC DONOR ID,CBC.MCH,DAY
SAMPLE ID,,,
S001_D10,S001,29.080675,10
S001_D23,S001,29.080675,23
S001_D42,S001,29.080675,42
S002_D10,S002,30.669145,10
S002_D23,S002,30.669145,23
...,...,...,...
S648_D42,S648,28.158845,42
S649_D10,S649,31.839623,10
S649_D42,S649,31.839623,42


### Transform data to copy numbers and expected format

In [17]:
# Convert Da to kDa
df_uniprot_to_mw = df_protein_data["Mass"] / 1000

df_concentrations = df_proteomics_final.copy()
# Sum intensities and convert to pmol / mgDW sample, then to nmol / gDW sample
df_concentrations = (
    df_concentrations.apply(lambda x: x / x.sum(), axis=1) / df_uniprot_to_mw
) * 1e6

# Conversion to copy numbers
df_copy_numbers = (df_concentrations * 1e-9).mul(
    df_MCH_final * 1e-12, axis=0
) * AVOGADRO_NUMBER
df_copy_numbers

,A0A075B6I0,A0A075B6I9,A0A075B6J9,A0A075B6K4,A0A075B6K5,A0A075B6R2,A0A075B6S5,A0A075B6S9,A0A087WSY6,A0A0A0MRZ8,...,Q9Y639,Q9Y666,Q9Y696,Q9Y6B6,Q9Y6B7,Q9Y6E0,Q9Y6I3,Q9Y6M4,Q9Y6M5,Q9Y6R7
SAMPLE ID,,,,,,,,,,,,,,,,,,,,,
S001_D10,114518.643899,63113.250142,17612.721227,1033.70001,250738.681069,0.000000,8828.312669,0.000000,323739.751835,61753.957120,...,1392.454161,0.000000,19954.371068,74758.901488,0.000000,7907.492758,4904.162257,0.000000,8864.838390,4551.934758
S001_D23,83733.413360,17891.485645,0.000000,0.00000,225441.984974,0.000000,28274.865365,138922.828686,323562.039664,63871.782876,...,933.842764,0.000000,11790.361238,0.000000,0.000000,8105.404114,31344.509782,5002.350276,8154.842476,4346.295603
S001_D42,124576.860801,51427.456139,64512.914721,0.00000,232622.665201,0.000000,26229.595680,0.000000,307083.059312,85379.086435,...,2177.042354,161.034294,11937.380407,80634.321987,1094.445937,11943.564971,4265.520169,5596.879103,6604.146775,4345.806113
S002_D10,0.000000,0.000000,0.000000,0.00000,246593.949891,10651.008268,22270.446851,0.000000,0.000000,11125.538105,...,3323.982341,0.000000,21133.854588,46229.818435,0.000000,11718.406394,0.000000,0.000000,23386.405820,5267.201961
S002_D23,52980.619738,0.000000,415956.112710,0.00000,443011.906351,0.000000,0.000000,0.000000,0.000000,0.000000,...,2508.714731,4267.543219,17838.394210,6541.070445,0.000000,16618.381242,0.000000,0.000000,24665.939736,1729.727805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mean_G6PD_V68M_1,117065.857775,28996.559803,55539.041460,1249.77806,202839.773898,2594.516292,21972.581427,34335.400717,95107.271898,60532.961910,...,1624.801723,235.763250,14786.680848,23782.626472,332.319954,10491.933458,4563.014979,58092.821594,10448.155499,2429.421989
Mean_G6PD_V68M_2,139180.644210,7950.349642,85820.061479,0.00000,639072.385934,10701.784802,43976.044731,41722.885395,88747.465745,116737.419453,...,2025.980309,253.815695,14385.658987,37387.074842,107.794642,11594.126106,2486.340090,96799.575979,11731.980883,3071.605902
Median_G6PD_V68M_0,85338.051512,0.000000,28111.153528,0.00000,305781.973449,1090.676767,19682.020245,0.000000,0.000000,69745.650159,...,2117.333048,0.000000,15613.816087,18619.522959,0.000000,11041.901353,3025.090168,0.000000,11632.019272,2984.929299


### Export absolute quantitative data and metadata per sample

In [18]:
dataframes_dict = {
    "ProteinData": df_protein_data,
    "ProteinIntensities": df_proteomics_final,
    "ProteinConcentrations": df_concentrations,
    "ProteinCopyNumbers": df_copy_numbers,
    "MCH": df_MCH_final,
    "Metadata": df_metadata,
}
for data_type, df in dataframes_dict.items():
    df.to_csv(
        processed_data_dirpath / f"{dataset_name}_{data_type}.tsv", sep="\t", index=True
    )
    print(f"Saved data for {data_type}")

Saved data for ProteinData
Saved data for ProteinIntensities
Saved data for ProteinConcentrations
Saved data for ProteinCopyNumbers
Saved data for MCH
Saved data for Metadata
